In [1]:
import datetime as dt
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

# Read the data for year 2010:
df_ = pd.read_excel("../datasets/online_retail_II.xlsx", sheet_name="Year 2009-2010")


In [2]:
df=df_.copy()

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [3]:
df.shape

(525461, 8)

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,525461.00000,10.33767,107.42411,-9600.00000,1.00000,3.00000,10.00000,19152.00000
Price,525461.00000,4.68883,146.12691,-53594.36000,1.25000,2.10000,4.21000,25111.09000
Customer ID,417534.00000,15360.64548,1680.81132,12346.00000,13983.00000,15311.00000,16799.00000,18287.00000


In [5]:
# Number of missing values
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [6]:
df.dropna(inplace=True) # Drop missing values
df = df[~df["Invoice"].str.contains("C", na=False)] # Remove the canceled orders
df = df[(df['Quantity'] > 0)] # Remove quantities with nonpositive values
df = df[(df['Price'] > 0)] # Remove prices with nonpositive values

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [7]:
df["TotalPrice"] = df["Quantity"] * df["Price"] # Add TotalPrice column 

In [8]:
today_date = df["InvoiceDate"].max().to_pydatetime() + dt.timedelta(days=2) # Set today's date to two days after the most recent date in the data set
today_date #= dt.datetime.fromtimestamp(today_date)

datetime.datetime(2010, 12, 11, 20, 1)

In [9]:
# Compute recency, frequency and monetary values for each customer

rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda InvoiceDate: (today_date - InvoiceDate.max()).days,
                                     'Invoice': lambda Invoice: Invoice.nunique(),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})
                                     
rfm.columns = ['recency', 'frequency', 'monetary']

rfm.head()


,recency,frequency,monetary
Customer ID,,,
12346.00000,166,11,372.86000
12347.00000,4,2,1323.32000
12348.00000,75,1,222.16000
12349.00000,44,3,2671.14000
12351.00000,12,1,300.93000


In [10]:
# Get recency, frequency and monetary scores (between 1-5) and combine them toget the RF(M) score (M will be excluded)

#rfm["recency_score"] = pd.qcut(rfm['recency'], 5, labels=[5, 4, 3, 2, 1])
a=rfm['recency']
cc=pd.qcut(pd.Series(np.linspace(a.min(),a.max(),a.max()-a.min()+1)), 5, labels=[5, 4, 3, 2, 1]).set_axis(np.linspace(a.min(),a.max(),a.max()-a.min()+1)).to_dict()
rfm["recency_score"]=a.replace(cc)

#rfm["frequency_score"] = pd.qcut(rfm['frequency'].rank(method="first"), 5, labels=[1, 2, 3, 4, 5])
a=rfm['frequency']
cc=pd.qcut(pd.Series(np.linspace(a.min(),a.max(),a.max()-a.min()+1)), 5, labels=[1, 2, 3, 4, 5]).set_axis(np.linspace(a.min(),a.max(),a.max()-a.min()+1)).to_dict()
rfm["frequency_score"]=a.replace(cc)

rfm["monetary_score"] = pd.qcut(rfm['monetary'], 5, labels=[1, 2, 3, 4, 5])
rfm["RFM_SCORE"] = (rfm['recency_score'].astype(str) + rfm['frequency_score'].astype(str))

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.00000,166,11,372.86000,3,1,2,31
12347.00000,4,2,1323.32000,5,1,4,51
12348.00000,75,1,222.16000,5,1,1,51
12349.00000,44,3,2671.14000,5,1,5,51
12351.00000,12,1,300.93000,5,1,2,51


In [11]:
rfm[["RFM_SCORE"]].value_counts()

RFM_SCORE
51           2673
41            618
31            407
21            356
11            231
52             15
53              8
54              2
42              1
55              1
dtype: int64

In [12]:
# Map RF scores to segments such as:

segment_map = {
    r'51': 'new',
    r'41': 'promising',
    r'31': 'potential_inactive',
    r'[1-2][1-2]': 'inactive',
    r'[4-5][2-3]': 'potential_loyal',
    r'5[4-5]': 'loyal'
}

rfm['segment'] = rfm['RFM_SCORE'].replace(segment_map, regex=True)

rfm.head()

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.00000,166,11,372.86000,3,1,2,31,potential_inactive
12347.00000,4,2,1323.32000,5,1,4,51,new
12348.00000,75,1,222.16000,5,1,1,51,new
12349.00000,44,3,2671.14000,5,1,5,51,new
12351.00000,12,1,300.93000,5,1,2,51,new


In [13]:
# Means of recency, frequency, and monetary; and the number of customers in each segment:
rfm[["segment", "recency", "frequency", "monetary"]].groupby("segment").agg(["mean", "count"])

recency       frequency          monetary      
                        mean count      mean count        mean count
segment                                                             
inactive           291.76491   587   1.43782   587   555.20234   587
loyal                4.33333     3 168.00000     3 75412.03333     3
new                 30.91470  2673   5.09502  2673  2134.61411  2673
potential_inactive 187.83292   407   1.99263   407   703.81821   407
potential_loyal     12.12500    24  69.25000    24 66342.69883    24
promising          108.07767   618   2.86893   618  1125.17978   618

In [14]:
# See loyal customers:
rfm[rfm["segment"] == "loyal"]

,recency,frequency,monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12748.00000,2,144,22879.66000,5,4,5,54,loyal
14911.00000,2,205,152147.57000,5,5,5,55,loyal
17850.00000,9,155,51208.87000,5,4,5,54,loyal
